In [1]:
%config IPCompleter.greedy=True # We need it to ensure the autocompletion feature works
import pandas as pd 
%matplotlib inline
import datetime # will be used later for formating date

## Merging of states data

* Aim:

    To merge all the states data into one single dataframe in the format:  
    ItemName | StateName | Jan 2016 | Feb 2019 | ...
  

* Steps:

 1. get the names of all the sheets
 2. check how the sheet looks
 3. create an empty data frame
 4. Loop through the sheet
 5. remove unwanted rows
 6. set the column names
 7. remove unwanted columns
 8. create a new column `State` with the sheet name as its value
 9. Concatenate the ouput of step 4 with the step 3 dataframe
 

In [3]:
## 2016 Data

In [2]:
workbook1 = pd.read_excel('.\\Raw Data\\Selected food price watch mar 2017- proshare.xlsx', sheet_name=None) #read all sheets
workbook1.keys() #check all the sheet names

odict_keys(['NATIONAL', 'ABIA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA', 'CROSS RIVER', 'DELTA', 'EDO', 'RIVERS', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'TARABA', 'YOBE', 'ABUJA', 'BENUE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'OGUN', 'ONDO', 'OSUN', 'OYO', 'JIGAWA', 'KADUNA', 'KANO', 'KATSINA', 'KEBBI', 'ZAMFARA', 'SOKOTO', 'Sheet1', 'Sheet2'])

In [5]:
to_delete = {'NATIONAL','Sheet1', 'Sheet2'} #unwanted sheets
for k in to_delete:
    workbook1.pop(k, None)

In [75]:
random_sheet = pd.read_excel('.\\Raw Data\\Selected food price watch mar 2017- proshare.xlsx', sheet_name="RIVERS") #read all sheets


In [42]:
random_sheet = pd.read_excel('.\\Raw Data\\Selected food price watch mar 2017- proshare.xlsx', sheet_name="RIVERS") #read all sheets
random_sheet.columns = random_sheet.iloc[1] #setting the colum name
random_sheet.drop([0,1],inplace=True) #deleting the first two rows
random_sheet=random_sheet[:-1] # eliminating the last one unwanted row
unit_of_measure = random_sheet.iloc[:,0:2].set_index([random_sheet.columns[0]])
unit_of_measure = unit_of_measure.to_dict()['Unit of Measurement']
unit_of_measure

{'Agric eggs medium size': '1 Dozen',
 'Agric eggs(medium size price of one)': '1 Piece',
 'Beans brown,sold loose': '1Kg',
 'Beans:white black eye. sold loose': '1Kg',
 'Beef Bone in': '1Kg',
 'Beef,boneless': '1Kg',
 'Bread sliced 500g': '1 loaf',
 'Bread unsliced 500g': '1 loaf',
 'Broken Rice (Ofada)': '1Kg',
 'Chicken Feet': '1Kg',
 'Chicken Wings': '1Kg',
 'Evaporated tinned milk carnation 170g': '1 unit',
 'Evaporated tinned milk(peak), 170g': '1 unit',
 'Frozen chicken': '1 unit',
 'Gari white,sold loose': '1Kg',
 'Gari yellow,sold loose': '1Kg',
 'Mudfish (aro) fresh': '1Kg',
 'Mudfish : dried': '1Kg',
 'Onion bulb': '1Kg',
 'Rice agric sold loose': '1Kg',
 'Rice local sold loose': '1Kg',
 'Rice Medium Grained': '1Kg',
 'Rice,imported high quality sold loose': '1Kg',
 'Tomato': '1Kg',
 'Yam tuber': '1Kg'}

In [36]:
merged_prices1 = pd.DataFrame()
for name, sheet in workbook1.items():
    rf_data = sheet.copy(); #getting copy of the real sheet
    try:
        rf_data.columns = rf_data.iloc[1] #setting the colum name
        rf_data.drop([0,1],inplace=True) #deleting the first two rows
        rf_data=rf_data[:-1] # eliminating the last one unwanted row
        rf_data= rf_data[rf_data.columns[0:-5]] #eliminating the last five unwanted columns
        
        rf_data.drop(["Unit of Measurement"], axis=1,inplace=True) #will be saved later
        rf_data.set_index([rf_data.columns[0]], inplace=True)
        rf_data=pd.concat([rf_data],keys=[name])
        merged_prices1 = pd.concat([merged_prices1,rf_data], sort=True)
        
        
        #print("SUCCESS: " + name)
    except:
        print("ERROR: "+ name)
        raise Exception

# creating a mapper for ensuring that only the datetime types get formated 
mapper = lambda x:x.strftime("%b %Y") if isinstance(x,datetime.datetime) else x
merged_prices1.columns = merged_prices1.columns.map(mapper)
merged_prices1

1                                             Jan 2016 Feb 2016 Mar 2016  \
        ItemLabels                                                         
ABIA    Agric eggs medium size                  333.81  333.571  343.957   
        Agric eggs(medium size price of one)   29.4643  29.8214       30   
        Beans brown,sold loose                 281.151  288.889  289.316   
        Beans:white black eye. sold loose      264.881  265.575  273.771   
        Beef Bone in                           796.561  798.405  757.391   
        Beef,boneless                          1029.08  1158.49  1119.51   
        Bread sliced 500g                      261.458  250.556  278.125   
        Bread unsliced 500g                    241.389  272.917  247.083   
        Broken Rice (Ofada)                     350.34   350.76  351.181   
        Chicken Feet                           652.052  720.832  724.537   
        Chicken Wings                          710.499  841.667      825   
        Evaporated tinned milk carnation 170g   125.45  125.601  125.751   
        Evaporated tinned milk(peak), 170g     133.492  134.065  138.483   
        Frozen chicken                         1144.94  1128.57  1204.55   
        Gari white,sold loose                  114.543   118.49  126.825   
        Gari yellow,sold loose                 120.212  124.342   137.87   
        Mudfish (aro) fresh                     780.23  781.166  782.104   
        Mudfish : dried                        2154.36  1925.73  1995.03   
        Onion bulb                             207.884  203.169  174.759   
        Rice agric sold loose                  253.477  243.099  250.426   
        Rice local sold loose                  157.199  175.118  215.271   
        Rice Medium Grained                    191.347  189.378   243.76   
        Rice,imported high quality sold loose  220.816  222.486  293.964   
        Tomato                                 244.533  197.865  180.912   
        Yam tuber                              100.139  129.604   128.66   
ANAMBRA Agric eggs medium size                 340.042  343.333  356.637   
        Agric eggs(medium size price of one)   29.5833  29.7917       30   
        Beans brown,sold loose                 238.027  242.947  264.445   
        Beans:white black eye. sold loose      223.868   223.51  246.077   
        Beef Bone in                           781.128  791.617  801.139   
...                                                ...      ...      ...   
ZAMFARA Rice local sold loose                  143.441  164.896  181.254   
        Rice Medium Grained                    133.938   185.86  148.955   
        Rice,imported high quality sold loose   216.87  229.861  253.156   
        Tomato                                 190.802  198.055  151.214   
        Yam tuber                              171.803  143.145  152.767   
SOKOTO  Agric eggs medium size                     360   362.34      360   
        Agric eggs(medium size price of one)        31  31.2015     31.5   
        Beans brown,sold loose                 182.695  183.883   198.72   
        Beans:white black eye. sold loose      136.161  137.046  153.981   
        Beef Bone in                           869.607   875.26      800   
        Beef,boneless                              975  981.337     1130   
        Bread sliced 500g                       218.75  220.172   256.25   
        Bread unsliced 500g                     206.25  207.591      250   
        Broken Rice (Ofada)                    148.852  149.819  163.795   
        Chicken Feet                           514.583      500  904.167   
        Chicken Wings                              650      650  666.667   
        Evaporated tinned milk carnation 170g      120   120.78    112.5   
        Evaporated tinned milk(peak), 170g      126.25  127.071      130   
        Frozen chicken                            1500  1509.75     1250   
        Gari white,sold loose                   

In [104]:
merged_prices1.to_csv(".\\Output\\merged_prices1.csv") # Save as a CSV file

In [ ]:
# 2017-Jul 2019 Data

In [7]:
workbook2 = pd.read_excel('.\\Raw Data\\SELECTED_FOOD_July_2019.xlsx', sheet_name=None) #read all sheets
workbook2.keys() #check all the sheet names

odict_keys(['ABIA', 'ABUJA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA', 'CROSS RIVER', 'DELTA', 'EDO', 'RIVERS', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'TARABA', 'YOBE', 'BENUE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'ONDO', 'OGUN', 'OSUN', 'OYO', 'JIGAWA', 'KADUNA', 'KANO', 'KATSINA', 'KEBBI', 'ZAMFARA', 'SOKOTO', 'NATIONAL'])

In [8]:
to_delete = {'NATIONAL','Sheet1', 'Sheet2'} #unwanted sheets
for k in to_delete:
    workbook2.pop(k, None)

In [37]:
merged_prices2 = pd.DataFrame()
for name, sheet in workbook2.items():
    rf_data = sheet.copy(); #getting copy of the real sheet
    try:
        rf_data.columns = rf_data.iloc[0] #setting the colum name
        rf_data.drop([0],inplace=True) #deleting the first row
        rf_data=rf_data[:-2] # eliminating the last two unwanted row
        rf_data= rf_data[rf_data.columns[0:-2]] #eliminating the last two unwanted columns
        
        #rf_data.drop(["Unit of Measurement"], axis=1,inplace=True) #will be saved later
        rf_data.set_index([rf_data.columns[0]], inplace=True)
        rf_data=pd.concat([rf_data],keys=[name])
        merged_prices2 = pd.concat([merged_prices2,rf_data], sort=True)        
        
        #print("SUCCESS: " + name)
    except:
        #print("ERROR: "+ name)
        raise Exception

# creating a mapper for ensuring that only the datetime types get formated 
mapper = lambda x:x.strftime("%b %Y") if isinstance(x,datetime.datetime) else x
merged_prices2.columns = merged_prices2.columns.map(mapper)
merged_prices2

0                                                Jan 2017 Feb 2017 Mar 2017  \
       ItemLabels                                                             
ABIA   Agric eggs medium size                     459.977   485.81  519.565   
       Agric eggs(medium size price of one)       45.1071  44.5136     46.4   
       Beans brown,sold loose                      471.22   436.72  415.625   
       Beans:white black eye. sold loose          420.764   375.09  400.242   
       Beef Bone in                               1107.67  955.095  981.258   
       Beef,boneless                               1378.4  1304.72  1129.16   
       Bread sliced 500g                          334.258  340.278      340   
       Bread unsliced 500g                        291.792  289.167  286.364   
       Broken Rice (Ofada)                            472      450  462.645   
       Chicken Feet                                774.76   731.25  735.714   
       Chicken Wings                               1060.1      900      920   
       Evaporated tinned milk carnation 170g       158.16      140      145   
       Evaporated tinned milk(peak), 170g         177.027      175    175.8   
       Frozen chicken                             1459.48  1509.72  1488.24   
       Gari white,sold loose                      270.932   267.46  286.957   
       Gari yellow,sold loose                     310.091  284.407  308.308   
       Mudfish (aro) fresh                         904.04  1015.22  1059.63   
       Mudfish : dried                            2642.63  2907.83  2984.74   
       Onion bulb                                 253.459  208.272  267.403   
       Rice agric sold loose                      440.704  393.008  386.209   
       Rice local sold loose                      363.704  382.846  366.182   
       Rice Medium Grained                         443.03  423.176  390.303   
       Rice,imported high quality sold loose       460.38  447.742  427.636   
       Tomato                                     309.656  306.309  333.333   
       Yam tuber                                  183.684  172.489  179.101   
       Dried Fish Sardine                          1759.8  1840.48  1524.31   
       Iced Sardine                                886.66   935.02  839.435   
       Irish potato                                249.69  293.965  299.075   
       Sweet potato                               116.168  116.008   112.68   
       Tilapia fish (epiya) fresh                  894.65  895.634  1052.63   
...                                                   ...      ...      ...   
SOKOTO Frozen chicken                             1640.48     1600  1700.32   
       Gari white,sold loose                      215.505  214.167  269.238   
       Gari yellow,sold loose                      291.53  275.746  302.407   
       Mudfish (aro) fresh                         876.59     1000     1000   
       Mudfish : dried                            1295.67  1221.43     1240   
       Onion bulb                                 244.918  210.198  227.302   
       Rice agric sold loose                      275.155   275.67  275.862   
       Rice local sold loose                      248.693   264.24  308.405   
       Rice Medium Grained                        266.845   270.88  275.862   
       Rice,imported high quality sold loose      356.185  325.556  319.048   
       Tomato                                     270.797  268.036  257.483   
       Yam tuber                                   259.25  267.389  285.208   
       Dried Fish Sardine                         1375.51  1348.39     1350   
       Iced Sardine                                 730.6      750   761.36   
       Irish potato                                184.58  185.699   188.35   
       Sweet potato                               93.5683  94.9404   93.405   
       Tilapia fish (epiya) fresh                 877.635      885    897.5   
       Titus:frozen                               

In [45]:
state_food_prices = merged_prices1.join(merged_prices2, how="outer")
item_with_unit={}
for key in unit_of_measure.keys():
     item_with_unit[key] = key + " / " + unit_of_measure[key]
state_food_prices = state_food_prices.reset_index().rename(columns={"level_0":"States"})
state_food_prices.replace({'ItemLabels':item_with_unit}, inplace=True)
state_food_prices

In [48]:
state_food_prices.to_csv(".\\Output\\state_food_prices.csv", index=False) # Save as a CSV file